In [ ]:
import json
import os
from tqdm import tqdm
import joblib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df = pd.read_pickle('jq_features.pkl')

データフレームdfがあります。  
dfにはDisclosedDateというカラムがあります。これが今日のちょうど一年前までを学習用（一部を検証用に分割）、それ以降をテスト用として使用します。  
dfには'ftn.'で始まるカラムがあり、これらは数値特徴量です。NaNを欠損値として処理します。  
dfには'ftc.'で始まるカラムがあり、これらはカテゴリ特徴量です。Noneを欠損値として処理します。  
これらの特徴量をxgboostで学習します。ラベルは'label'というカラム（数値）を使用します。前進法で特徴量選択を行います。アーリーストッピングも実装します。  

In [ ]:
# 1. データの分割
def split_data_by_date(df):
    """
    今日の1年前を基準に学習用とテスト用にデータを分割
    """
    today = datetime.now()
    one_year_ago = today - timedelta(days=365)
    
    # DisclosedDateをdatetime型に変換
    df['DisclosedDate'] = pd.to_datetime(df['DisclosedDate'])
    
    # labelのNaNを0に置換
    df['label'] = df['label'].fillna(0)
    print(f"labelのNaN件数: {df['label'].isna().sum()} 件 (0に置換済み)")
    
    # 学習用とテスト用に分割
    train_df = df[df['DisclosedDate'] <= one_year_ago].copy()
    test_df = df[df['DisclosedDate'] > one_year_ago].copy()
    
    print(f"学習用データ: {len(train_df)} 件 ({train_df['DisclosedDate'].min()} ~ {train_df['DisclosedDate'].max()})")
    print(f"テスト用データ: {len(test_df)} 件 ({test_df['DisclosedDate'].min()} ~ {test_df['DisclosedDate'].max()})")
    
    # labelの分布を確認
    print(f"\n学習用ラベルの統計:")
    print(f"  平均: {train_df['label'].mean():.4f}")
    print(f"  標準偏差: {train_df['label'].std():.4f}")
    print(f"  最小値: {train_df['label'].min():.4f}")
    print(f"  最大値: {train_df['label'].max():.4f}")
    print(f"  ゼロの割合: {(train_df['label'] == 0).mean():.2%}")
    
    return train_df, test_df

In [ ]:
# 1. データを時系列で分割
train_df, test_df = split_data_by_date(df)

In [ ]:
# 2. 特徴量の前処理
def preprocess_features(train_df, test_df):
    """
    数値特徴量とカテゴリ特徴量の前処理
    """
    import numpy as np
    
    # 特徴量カラムの識別
    numeric_cols = [col for col in train_df.columns if col.startswith('ftn.')]
    categorical_cols = [col for col in train_df.columns if col.startswith('ftc.')]
    
    print(f"数値特徴量: {len(numeric_cols)} 個")
    print(f"カテゴリ特徴量: {len(categorical_cols)} 個")
    
    # 処理後のデータフレーム
    train_processed = train_df.copy()
    test_processed = test_df.copy()
    
    # 数値特徴量の無限大処理（inf, -infをNaNに変換）
    # XGBoostはNaNを自動的にmissingとして扱うため、fillnaは不要
    for col in numeric_cols:
        # 正の無限大と負の無限大をNaNに置換
        train_processed[col] = train_processed[col].replace([np.inf, -np.inf], np.nan)
        test_processed[col] = test_processed[col].replace([np.inf, -np.inf], np.nan)
    
    # カテゴリ特徴量の処理
    label_encoders = {}
    for col in categorical_cols:
        # Noneを'missing'に置換
        train_processed[col] = train_processed[col].fillna('missing')
        test_processed[col] = test_processed[col].fillna('missing')
        
        # Label Encoding
        le = LabelEncoder()
        # 学習データで fit
        train_values = train_processed[col].astype(str)
        le.fit(train_values)
        
        # 変換
        train_processed[col] = le.transform(train_values)
        
        # テストデータの未知のカテゴリを処理
        test_values = test_processed[col].astype(str)
        test_values_encoded = []
        for val in test_values:
            if val in le.classes_:
                test_values_encoded.append(le.transform([val])[0])
            else:
                test_values_encoded.append(-1)  # 未知のカテゴリ
        test_processed[col] = test_values_encoded
        
        label_encoders[col] = le
    
    return train_processed, test_processed, numeric_cols + categorical_cols

In [ ]:
# 2. 特徴量の前処理
train_processed, test_processed, feature_cols = preprocess_features(train_df, test_df)

In [ ]:
# 3. 学習データをさらに訓練用と検証用に分割
X_train_full = train_processed[feature_cols]
y_train_full = train_processed['label']

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, 
    test_size=0.2, 
    random_state=42
)

print(f"\n訓練データ: {len(X_train)} 件")
print(f"検証データ: {len(X_val)} 件")

In [ ]:
# 3. 効率化された特徴量選択（後退法）
def train_xgboost_light(X_train, y_train, X_val, y_val, features, num_boost_round=100):
    """
    軽量版のXGBoost学習（特徴量選択用）
    """
    dtrain = xgb.DMatrix(X_train[features], label=y_train)
    dval = xgb.DMatrix(X_val[features], label=y_val)
    
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 3,
        'gamma': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,
        'tree_method': 'hist',
        'device': 'cuda',
        'seed': 42,
        'verbosity': 0
    }
    
    evals_result = {}
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dval, 'val')],
        evals_result=evals_result,
        verbose_eval=False
    )
    
    val_score = evals_result['val']['rmse'][-1]
    return val_score, model


def remove_all_nan_features(X_train, X_val, features):
    """
    すべてNaNの特徴量を除外する
    
    Parameters:
    -----------
    X_train : 訓練データ
    X_val : 検証データ
    features : チェックする特徴量のリスト
    
    Returns:
    --------
    valid_features : NaNでない有効な特徴量のリスト
    nan_features : すべてNaNだった特徴量のリスト
    """
    valid_features = []
    nan_features = []
    
    for feature in features:
        # 訓練データと検証データの両方でチェック
        train_all_nan = X_train[feature].isna().all() if feature in X_train.columns else True
        val_all_nan = X_val[feature].isna().all() if feature in X_val.columns else True
        
        if train_all_nan or val_all_nan:
            nan_features.append(feature)
        else:
            valid_features.append(feature)
    
    return valid_features, nan_features


def feature_selection_backward(X_train, y_train, X_val, y_val, all_features, 
                               min_features=50, patience=20, max_degradation=0.001,
                               batch_removal=True, improvement_ratio=0.5):

    """
    効率化された後退法による特徴量選択
    性能劣化が最小の特徴量から順に除外、劣化が小さい複数特徴量を同時除外可能
    
    Parameters:
    -----------
    X_train, y_train, X_val, y_val : 学習・検証データ
    all_features : 全特徴量のリスト
    min_features : 最小特徴量数（これ以下にはしない）
    patience : 改善が見られない連続回数の上限
    max_degradation : 許容される最大性能劣化（正の値）
    batch_removal : 複数特徴量同時除外を有効にするか
    improvement_ratio : 性能改善時にバッチに追加する特徴量の改善比率閾値
    """
    print("\n=== 効率化された後退法による特徴量選択開始 ===")
    
    # すべてNaNの特徴量を除外
    valid_features, nan_features = remove_all_nan_features(X_train, X_val, all_features)
    
    if nan_features:
        print(f"\n警告: {len(nan_features)}個の特徴量がすべてNaNのため除外されました")
        print(f"  除外された特徴量の例（最大10個）: {nan_features[:10]}")
        if len(nan_features) > 10:
            print(f"  ... 他 {len(nan_features)-10}個")
    
    print(f"\n有効な特徴量数: {len(valid_features)} (元: {len(all_features)})")
    print(f"最小特徴量数: {min_features}")
    print(f"Patience: {patience}")
    print(f"許容最大劣化: {max_degradation}")
    print(f"バッチ除外: {'有効' if batch_removal else '無効'}")
    
    # 初期状態：全ての有効な特徴量を使用
    selected_features = valid_features.copy()
    
    # 初期スコアを計算
    print(f"\n初期特徴量数: {len(selected_features)}")
    print("初期スコアを計算中...")
    best_score, _ = train_xgboost_light(X_train, y_train, X_val, y_val, selected_features)
    print(f"初期スコア: {best_score:.6f}")
    
    patience_counter = 0
    removed_count = 0
    
    # 後退法による特徴量選択
    print("\n--- 効率化された後退法による特徴量選択 ---")
    
    while len(selected_features) > min_features and patience_counter < patience:
        candidate_scores = []
        
        print(f"\n特徴量除外候補を評価中... (現在の特徴量数: {len(selected_features)})")
        
        # 各特徴量を除外した場合の影響を評価
        for feature in tqdm(selected_features, desc="候補評価"):
            current_features = [f for f in selected_features if f != feature]
            
            # 安全チェック: 除外後も有効な特徴量が残るか
            if len(current_features) < min_features:
                continue
            
            try:
                score, _ = train_xgboost_light(X_train, y_train, X_val, y_val, current_features)
                degradation = score - best_score  # 正の値が除外による性能劣化
                candidate_scores.append((feature, score, degradation))
            except Exception as e:
                print(f"特徴量 {feature} の除外評価でエラー: {e}")
                continue
        
        if not candidate_scores:
            print("評価可能な候補がありません")
            break
        
        # 除外による性能劣化が小さい順にソート（劣化が小さい = 除外しても影響が少ない）
        candidate_scores.sort(key=lambda x: x[2])
        
        # 最小劣化
        min_degradation = candidate_scores[0][2]
        
        # 性能が改善または許容範囲内の劣化の場合
        if min_degradation <= max_degradation:
            features_to_remove = []
            
            # 複数特徴量同時除外の実行
            if batch_removal and min_degradation < 0:  # 性能改善時のみバッチ除外を検討
                # 十分に性能を改善した特徴量を特定
                improvement_threshold = abs(min_degradation) * improvement_ratio
                
                improvement_features = [
                    (feature, score, degradation) 
                    for feature, score, degradation in candidate_scores 
                    if degradation < 0 and abs(degradation) >= improvement_threshold
                ]
                
                print(f"\n性能改善候補: {len(improvement_features)}個")
                print(f"  改善閾値: {improvement_threshold:.6f}")
                for i, (feature, score, degradation) in enumerate(improvement_features[:5]):
                    print(f"    {i+1}. {feature}: 改善 {abs(degradation):.6f} (スコア: {score:.6f})")
                if len(improvement_features) > 5:
                    print(f"    ... 他 {len(improvement_features)-5}個")
                
                # 改善特徴量での同時除外を検証
                if improvement_features:
                    batch_remove_features = [f[0] for f in improvement_features]
                    test_features = [f for f in selected_features if f not in batch_remove_features]
                    
                    if len(test_features) >= min_features:
                        try:
                            batch_score, _ = train_xgboost_light(X_train, y_train, X_val, y_val, test_features)
                            batch_degradation = batch_score - best_score
                            
                            if batch_degradation < 0:  # さらに性能改善を確認
                                selected_features = test_features
                                best_score = batch_score
                                patience_counter = 0
                                removed_count += len(batch_remove_features)
                                
                                print(f"バッチ除外成功: {len(batch_remove_features)}個の特徴量を同時除外")
                                print(f"  除外特徴量: {batch_remove_features[:5]}")
                                if len(batch_remove_features) > 5:
                                    print(f"  ... 他 {len(batch_remove_features)-5}個")
                                print(f"  現在の特徴量数: {len(selected_features)}")
                                print(f"  スコア: {best_score:.6f} (改善幅: {abs(batch_degradation):.6f})")
                                continue
                        except Exception as e:
                            print(f"バッチ除外でエラー: {e}")
            
            # 単一特徴量除外
            worst_feature, worst_score, worst_degradation = candidate_scores[0]
            
            if worst_degradation <= max_degradation:
                selected_features.remove(worst_feature)
                best_score = worst_score
                patience_counter = 0
                removed_count += 1
                
                print(f"単一特徴量除外: {worst_feature}")
                print(f"  現在の特徴量数: {len(selected_features)}")
                print(f"  スコア: {best_score:.6f} (劣化: {worst_degradation:.6f})")
                
                # 性能が改善した場合
                if worst_degradation < 0:
                    print(f"  ※性能が改善しました！ (改善幅: {abs(worst_degradation):.6f})")
            else:
                patience_counter += 1
                print(f"全ての候補で劣化が許容範囲外 (最小劣化: {worst_degradation:.6f} > {max_degradation:.6f})")
                print(f"patience: {patience_counter}/{patience}")
        else:
            patience_counter += 1
            print(f"全ての候補で劣化が許容範囲外 (最小劣化: {min_degradation:.6f} > {max_degradation:.6f})")
            print(f"patience: {patience_counter}/{patience}")
    
    print(f"\n=== 後退法による特徴量選択完了 ===")
    print(f"最終選択特徴量数: {len(selected_features)} (除外数: {removed_count})")
    print(f"最終スコア: {best_score:.6f}")
    if nan_features:
        print(f"事前除外されたNaN特徴量数: {len(nan_features)}")
    
    # 除外された特徴量のリストも返す
    removed_features = [f for f in valid_features if f not in selected_features]
    
    return selected_features, best_score, removed_features

In [ ]:

# 特徴量選択の実行
print(f"元の特徴量数: {len(feature_cols)}")
selected_features, selection_score = feature_selection_backward(
    X_train, y_train, X_val, y_val, feature_cols
)

print(f"\n選択された特徴量:")
for i, feature in enumerate(selected_features, 1):
    print(f"  {i:2d}. {feature}")


In [ ]:
# 4. XGBoostモデルの学習（アーリーストッピング付き）
def train_xgboost_with_early_stopping(X_train, y_train, X_val, y_val, selected_features):
    """
    選択された特徴量でXGBoostモデルを学習
    """
    print("\n=== XGBoostモデルの学習（アーリーストッピング付き） ===")
    
    # DMatrix形式に変換
    dtrain = xgb.DMatrix(X_train[selected_features], label=y_train)
    dval = xgb.DMatrix(X_val[selected_features], label=y_val)
    
    # パラメータ設定
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 8,
        'learning_rate': 0.05,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 3,
        'gamma': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 1.0,
        'tree_method': 'hist',
        'device': 'cuda',
        'seed': 42
    }
    
    # アーリーストッピング付きで学習
    evals_result = {}
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dval, 'val')],
        early_stopping_rounds=50,
        evals_result=evals_result,
        verbose_eval=100
    )
    
    print(f"\n最適なラウンド数: {model.best_iteration}")
    print(f"最良の検証スコア (RMSE): {model.best_score:.4f}")
    
    return model, evals_result

In [ ]:
# 5. 選択された特徴量でモデルを再学習（アーリーストッピング付き）
final_model, evals_result = train_xgboost_with_early_stopping(
    X_train, y_train, 
    X_val, y_val, 
    selected_features
)

In [ ]:
import joblib

# モデルの保存
joblib.dump(final_model, 'jq_model.joblib')
joblib.dump(selected_features, 'jq_selected_features.joblib')

# 特徴量選択結果の保存
feature_selection_results = {
    'selected_features': selected_features,
    'final_score': selection_score,
    'feature_count': len(selected_features),
    'original_feature_count': len(feature_cols)
}

with open('jq_feature_selection_results.json', 'w', encoding='utf-8') as f:
    json.dump(feature_selection_results, f, indent=2, ensure_ascii=False)

print(f"\nモデルと特徴量選択結果を保存しました:")
print(f"  - モデル: jq_model.joblib")
print(f"  - 選択特徴量: jq_selected_features.joblib") 
print(f"  - 選択結果: feature_selection_results.json")

In [ ]:
# 5. モデルの評価
def evaluate_model(model, X_test, y_test, selected_features):
    """
    テストデータでモデルを評価
    """
    dtest = xgb.DMatrix(X_test[selected_features])
    y_pred = model.predict(dtest)
    
    # RMSEを計算（squared=Falseが使えない場合の代替方法）
    try:
        # 新しいバージョンのscikit-learn
        rmse = mean_squared_error(y_test, y_pred, squared=False)
    except TypeError:
        # 古いバージョンのscikit-learn：MSEの平方根を取る
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print("\n=== テストデータでの評価結果 ===")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    
    return {
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': y_pred
    }

In [ ]:
# 6. テストデータで評価
X_test = test_processed[feature_cols]
y_test = test_processed['label']

results = evaluate_model(final_model, X_test, y_test, selected_features)


In [ ]:
# 6. 特徴量の重要度を表示
def plot_feature_importance(model, selected_features, top_n=20):
    """
    特徴量の重要度を表示
    """
    importance = model.get_score(importance_type='gain')
    
    # DataFrameに変換
    importance_df = pd.DataFrame(
        [(f, importance.get(f'f{i}', 0)) for i, f in enumerate(selected_features)],
        columns=['feature', 'importance']
    )
    importance_df = importance_df.sort_values('importance', ascending=False).head(top_n)
    
    print(f"\n=== Top {top_n} 重要な特徴量 ===")
    for idx, row in importance_df.iterrows():
        print(f"{row['feature']}: {row['importance']:.2f}")
    
    return importance_df

In [ ]:
# 7. 特徴量の重要度を表示
importance_df = plot_feature_importance(final_model, selected_features)

In [ ]:
# 学習曲線の可視化（オプション）
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(evals_result['train']['rmse'], label='Train RMSE')
plt.plot(evals_result['val']['rmse'], label='Validation RMSE')
plt.xlabel('Boosting Rounds')
plt.ylabel('RMSE')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import seaborn as sns

x = y_test
y = results['predictions']

sns.scatterplot(x=x, y=y, s=5, color=".15")
sns.histplot(x=x, y=y, bins=50, pthresh=.1, cmap="mako")
sns.kdeplot(x=x, y=y, levels=5, color="w", linewidths=1)

In [ ]:
# 特徴量選択結果の詳細分析
def analyze_feature_selection_results(original_features, selected_features, final_model):
    """
    特徴量選択結果の詳細分析
    """
    print("\n" + "="*60)
    print("特徴量選択結果の詳細分析")
    print("="*60)
    
    # 基本統計
    print(f"元の特徴量数: {len(original_features)}")
    print(f"選択された特徴量数: {len(selected_features)}")
    print(f"削減率: {(1 - len(selected_features)/len(original_features))*100:.1f}%")
    
    # 特徴量タイプ別の分析
    original_numeric = [f for f in original_features if f.startswith('ftn.')]
    original_categorical = [f for f in original_features if f.startswith('ftc.')]
    selected_numeric = [f for f in selected_features if f.startswith('ftn.')]
    selected_categorical = [f for f in selected_features if f.startswith('ftc.')]
    
    print(f"\n特徴量タイプ別:")
    print(f"  数値特徴量: {len(original_numeric)} → {len(selected_numeric)} ({len(selected_numeric)/len(original_numeric)*100:.1f}%)")
    print(f"  カテゴリ特徴量: {len(original_categorical)} → {len(selected_categorical)} ({len(selected_categorical)/len(original_categorical)*100:.1f}%)")
    
    # 重要度の上位特徴量
    importance = final_model.get_score(importance_type='gain')
    importance_df = pd.DataFrame(
        [(f, importance.get(f'f{i}', 0)) for i, f in enumerate(selected_features)],
        columns=['feature', 'importance']
    )
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    print(f"\n最も重要な特徴量 Top 10:")
    for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
        feature_type = "数値" if row['feature'].startswith('ftn.') else "カテゴリ"
        print(f"  {i+1:2d}. {row['feature']} ({feature_type}): {row['importance']:.2f}")
    
    return importance_df

# 分析実行
importance_analysis = analyze_feature_selection_results(feature_cols, selected_features, final_model)


In [ ]:
def compare_performance_before_after_selection(X_train, y_train, X_val, y_val, 
                                              original_features, selected_features):
    """
    特徴量選択前後の性能比較
    """
    print("\n" + "="*60)
    print("特徴量選択前後の性能比較")
    print("="*60)
    
    # 元の特徴量での性能
    print("全特徴量でのモデル学習中...")
    original_score, _ = train_xgboost_light(X_train, y_train, X_val, y_val, original_features, num_boost_round=200)
    
    # 選択特徴量での性能
    print("選択特徴量でのモデル学習中...")
    selected_score, _ = train_xgboost_light(X_train, y_train, X_val, y_val, selected_features, num_boost_round=200)
    
    print(f"\n性能比較結果:")
    print(f"  全特徴量 ({len(original_features)}個): RMSE = {original_score:.4f}")
    print(f"  選択特徴量 ({len(selected_features)}個): RMSE = {selected_score:.4f}")
    print(f"  性能変化: {((selected_score - original_score) / original_score * 100):+.2f}%")
    print(f"  特徴量削減: {((len(original_features) - len(selected_features)) / len(original_features) * 100):.1f}%")
    
    return original_score, selected_score

# 性能比較実行
original_performance, selected_performance = compare_performance_before_after_selection(
    X_train, y_train, X_val, y_val, feature_cols, selected_features
)